In [2]:
import pandas as pd
train_df = pd.read_csv("data_/StackOverflow_2017-11to2018-05.csv", parse_dates=["CreationDate"])

In [4]:
pd.set_option('display.max_colwidth', -1)
train_df.tail()

,Id,CreationDate,Body,Title,Tags
42708,49245339,2018-03-12 22:24:07,"<p>I am using material tabs: <a href=""https://material.angular.io/components/tabs/overview"" rel=""noreferrer"">https://material.angular.io/components/tabs/overview</a></p>\n\n<p>I have a page with a tab view that I want to use to populate one custom component per tab with a click of the button. Assume I have two components with the selectors ""customcomponent1"" and ""customcomponent2"". I want to make it such that when I click a button, a new 'object' will be added to the list like the following:</p>\n\n<blockquote>\n <p>{'component_name':'customcomponent1'}</p>\n</blockquote>\n\n<p>and a new tab ""Tab2"" will dynamically be created with that component inside of it is if it were like this:</p>\n\n<pre><code> &lt;mat-tab-group&gt;\n &lt;mat-tab label=""Tab1""&gt;\n&lt;button (click)=""createNewTabWithComponent('customcomponent1')""&gt;&lt;/button&gt;\n &lt;/mat-tab&gt;\n &lt;mat-tab label=""Tab2""&gt;\n &lt;customcomponent1&gt;&lt;/customcomponent1&gt;\n &lt;/mat-tab&gt;\n &lt;/mat-tab-group&gt;\n</code></pre>\n\n<p>If I click on the button again... I get a new tab:</p>\n\n<pre><code>&lt;mat-tab-group&gt;\n &lt;mat-tab label=""Tab1""&gt;\n&lt;button (click)=""createNewTabWithComponent('customcomponent1')""&gt;&lt;/button&gt;\n &lt;/mat-tab&gt;\n &lt;mat-tab label=""Tab2""&gt;\n &lt;customcomponent1&gt;&lt;/customcomponent1&gt;\n &lt;/mat-tab&gt;\n &lt;mat-tab label=""Tab3""&gt;\n &lt;customcomponent2&gt;&lt;/customcomponent2&gt;\n &lt;/mat-tab&gt;\n &lt;/mat-tab-group&gt;\n</code></pre>\n\n<p>How do I accomplish this with angular2/4? I normally would use something like $compile, but i dont think angular2/4 has one. What I want to avoid is creating a bunch of tabs for each component (imagine I have 10 components, really don't want to create multiple mat-tab placeholders for every single component and setting a show/hide flag on each one) and 'hardcoding' the one component inside of it.</p>\n\n<p>This isn't really a problem I think that is specific to tabs, if the solution shows how to add a component 'dynamically' using the name of the selector (that a user types in a textbox and hits a button to 'add to a list', that would be a considered answer too.</p>\n\n<p>A sample I can think of is if there is a text box, and somebody can type in any string. If a string matches the name of a component, then that component programatically 'dynamically shown' on the screen as if it were part of the original template.</p>\n\n<p>Pseudocode of what I am looking for that I know does not exist (i think, but it would be nice if it did):</p>\n\n<pre><code>&lt;button (click)=""addNameOfComponentToListOfCustomComponentStrings('text-from-a-textbox')""&gt;Add Component&lt;/button&gt;\n\n&lt;div *ngFor=""let string_name_of_component in listofcustomcomponentstrings""&gt;\n &lt;{{string_name_of_component}}&gt;&lt;&lt;/{{string_name_of_component}}&gt;\n&lt;/div&gt;\n</code></pre>\n\n<p>Getting them into tabs would be a plus. If this is not doable, please post. Otherwise, please post.</p>\n\n<p>If there is maybe a workaround that could be done using some 'nested angular routing' where it pulls up the component based on the name of the component in ngFor, that may work too... am open to any ideas.</p>\n",How to add new tabs with components using the 'name/selector of the component' programmatically using angular 2/4?,<angular><material>
42709,49244585,2018-03-12 21:21:48,"<p>I have a class like that:</p>\n\n<pre><code>public class Student implements Serializable{\n private String name;\n private int age;\n private Image img;\n}\n</code></pre>\n\n<p>I store a few students in an ArrayList and write them to a file. When I restart the application I load them from that file again. However, the <code>Image img</code> variable is constantly changing. That means when I use <code>arrayList.contains(studentA)</code> it's not the same object anymore, so <code>arrayList.remove(studentA)</code> won't

In [10]:
import numpy as np
import pandas as pd

import datetime
from datetime import timedelta, date
import pickle

import re
import nltk
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import pairwise_distances

X_train = pickle.load(open('data_/X_train_tags.pkl', 'rb'))
main_tags = pickle.load(open('data_/main_tags.pkl', 'rb'))

title = "Multitenancy in ASP.NET Core 2.0+"
body = '<p><strong>Bakground:</strong>\nI want to develop a multi-tenant application in ASP.NET Core and have been looking into Ben Fosters <a href="https://github.com/saaskit/saaskit" rel="nofollow noreferrer">Saaskit</a> library which seems to provide good solutions for common problems in multitenancy applications.</p>\n\n<p><strong>Problem:</strong>\nThe SaasKit have a <code>UsePerTenant</code> method which is nice for doing different things per-request depending on current tenant.</p>\n\n<p>My goal is to use the <code>UsePerTenant</code> method combined with different <code>IOptions</code> objects injected via dependency injection. This can be used in the authentication middleware like</p>\n\n<pre><code>AddAuthentication().AddCookie(..).AddOpenIdConnect(...)\n</code></pre>\n\n<p>Which is configured in the <code>ConfigureServices</code> method in Startup.cs</p>\n\n<pre><code>public class Startup\n{\n // Use this method to add services to the container.\n public void ConfigureServices(IServiceCollection services)\n {\n ...\n }\n\n // Use this method to configure the HTTP request pipeline.\n public void Configure(IApplicationBuilder app)\n {\n ...\n }\n}\n</code></pre>\n\n<p>I can’t make the authentication middleware in ASP.NET 2.0+ use different <code>IOptions</code> objects per-request since the <code>ConfigureServices</code> method in the Startup.cs file only runs once every application startup and the <code>UsePerTenant</code> method should be used in the <code>Configure</code> method which is running for each incoming/outgoing request in the ASP.NET pipeline.</p>\n\n<p><strong>Question:</strong>\nHow to dynamically change cookie and OpenID Connect options in the <code>ConfigureServices</code> method based on current tenant?</p>\n'

title = 'Look up type synonyms in haskell'
body = '<p>Is there a way to look up what type synonyms actually mean? Is there perhaps some GHCi command that I can use to see if a given type is a synonym of something else?</p>\n'


df = pd.DataFrame()
df = df.append({'Title': title, 'Body': body}, ignore_index=True)

df['Body_length'] = df['Body'].str.len()
df['Title_length'] = df['Title'].str.len()
df['Text_code'] = df['Body'].apply(lambda x: re.findall('<pre>.*?</pre>',  x.replace('\n', ' ').replace('\t', ' ').replace("\'", ' ')))
df['Text_code'] = df['Text_code'].apply(lambda x: [l.replace("<pre><code>", '').replace("</code></pre>", '') for l in x])
df['Nb_code_block'] = df['Text_code'].apply(lambda x: len(x))
df['Text'] = df['Title'] + '. ' + df['Body']

def clean_code_link_html(raw):
    raw = raw.replace('\n', ' ')
    raw = raw.replace('\t', ' ')
    raw = raw.replace("\'", ' ')
    raw = re.sub('<pre>.*?</pre>', ' ', raw, flags=re.MULTILINE)
    raw = re.sub('<a[^>]+>.*?</a>', ' ', raw)
    raw = re.sub('\S*@\S*\s?', ' ', raw)
    raw = raw.lower()
    return BeautifulSoup(raw, 'html.parser').get_text()

df['Text_cleaned'] = df['Text'].apply(clean_code_link_html)
df['Text_cleaned'] = df['Text_cleaned'].apply(lambda x: x.replace('#', 'diiiese'))
df['Text_words'] = df.Text_cleaned.apply(nltk.word_tokenize)
df['Text_words'] = df['Text_words'].apply(lambda x: [word.replace('diiiese', '#') for word in x])
# restricting words to alpha + nonalpha_to_use words
df['Text_words'] = df['Text_words'].apply(lambda x: [word for word in x if re.search(r'^[a-z.+#-]', word)])
df['Text_words'] = df['Text_words'].apply(lambda x: [word for word in x 
                                                                 if ((word!='.') & (word!='+') & (word!='#') & (word!='-'))])
# removing stopwords
stopwords = nltk.corpus.stopwords.words('english')
df['Text_words'] = df['Text_words'].apply(lambda x: [word for word in x if word not in stopwords])
df['Text_maintags'] = df['Text_words'].apply(lambda x: [word for word in x if word in main_tags])
df['Text_words'] = df['Text_words'].apply(lambda x: [word for word in x if word not in main_tags])
# lemmatization
wnl = nltk.WordNetLemmatizer()
df['Text_words_lemma'] = df['Text_words'].apply(lambda x: [wnl.lemmatize(word) for word in x])

def all_postags(tokens):
    all_postags = [(token,pos) for token, pos in nltk.pos_tag(tokens)]
    return all_postags

df['Text_postags'] = df['Text_words_lemma'].apply(all_postags)

# selecting just nouns as aspiring keywords
def just_nouns(tokens):
    nouns = [token for token, pos in nltk.pos_tag(tokens) if pos == 'NN']
    return nouns

df['Text_final_words'] = df['Text_words_lemma'].apply(just_nouns)
# and we add the words preserved from lemmatization
df['Text_final_words'] = df['Text_final_words'] + df['Text_maintags']
df['Text_code_final_words'] = df.Text_code.apply(lambda x: ' '.join(x)).apply(nltk.word_tokenize)
df['Text_count'] = df['Text_final_words'].apply(lambda x: len(set(x)))

def lexical_diversity(text):
    if len(text) > 0:
        return len(set(text)) / len(text)
    else:
        return 0

df['Text_ld'] = df['Text_words'].apply(lexical_diversity)
df['Text_final_words_str'] = df['Text_final_words'].apply(lambda x: ' '.join(x))
df['Text_code_final_words_str'] = df['Text_code_final_words'].apply(lambda x: ' '.join(x))

count_vect = pickle.load(open('data_/count_vect_code.pkl', 'rb'))
df_counts = count_vect.transform(df['Text_code_final_words_str'])

lda = pickle.load(open('data_/lda_code.pkl', 'rb'))
df_lda = lda.transform(df_counts)

X_train_lda = pickle.load(open('data_/X_train_lda_code.pkl', 'rb'))
X_dist = pairwise_distances(X_train_lda, df_lda, metric='cosine')

def extract_tags_from_nearest_posts(col, post, nb_tags=10):
    tags_extracted = []
    if len(df.loc[post, 'Text_code_final_words']) > 0:
        for i in col.argsort()[:nb_tags]:
            if len(X_train.loc[i, 'Tags_final']) > 0:
                if (len(tags_extracted) < nb_tags):
                    tags_extracted.append(X_train.loc[i, 'Tags_final'][0])
    return tags_extracted

tags_array = []
for test_post in range(len(df)):
    tags_array.append(extract_tags_from_nearest_posts(X_dist[:, test_post], test_post))
    
df['LDA_tags_from_code'] = tags_array


count_vect = pickle.load(open('data_/count_vect_text.pkl', 'rb'))
df_counts = count_vect.transform(df['Text_final_words_str'])

lda = pickle.load(open('data_/lda_text.pkl', 'rb'))
df_lda = lda.transform(df_counts)

X_train_lda = pickle.load(open('data_/X_train_lda_text.pkl', 'rb'))
X_dist = pairwise_distances(X_train_lda, df_lda, metric='cosine')

def extract_tags_from_nearest_posts(col, post, nb_tags=10):
    tags_extracted = []
    if len(df.loc[post, 'Text_final_words']) > 0:
        for i in col.argsort()[:nb_tags]:
            if len(X_train.loc[i, 'Tags_final']) > 0:
                if (len(tags_extracted) < nb_tags):
                    tags_extracted.append(X_train.loc[i, 'Tags_final'][0])
    return tags_extracted

tags_array = []
for test_post in range(len(df)):
    tags_array.append(extract_tags_from_nearest_posts(X_dist[:, test_post], test_post))
    
df['LDA_tags_from_text'] = tags_array

def select_mode_tags(row):
    w_count = {}
    for word in (row['LDA_tags_from_code'] + row['LDA_tags_from_text']):
        if word in w_count:
            w_count[word] += 1
        else:
            w_count[word] = 1
    mode_words = sorted(w_count, key=w_count.get, reverse=True)
    mode_words_str = ' '.join(mode_words[:10])
    return mode_words_str
    
df['LDA_tags'] = df.apply(select_mode_tags, axis=1)
df['LDA_tags'] = df['LDA_tags'].apply(lambda x: x.split())

df['LDA_tags']

0    [c++, haskell, windows, perl6, f#]
Name: LDA_tags, dtype: object

In [11]:
df['LDA_tags_from_code'] + df['LDA_tags_from_text']

0    [haskell, c++, c++, c++, c++, windows, c++, perl6, f#, c++]
dtype: object